In [1]:
import networkx as nx
import numpy as np
from pgmpy.models.BayesianModel import BayesianModel
from pgmpy.factors.discrete import TabularCPD
from pgmpy.factors.discrete import State
import random as random

In [2]:
student = BayesianModel([('diff', 'grade'), ('intel', 'grade')])
cpd_d = TabularCPD('diff', 2, [[0.6], [0.4]])
cpd_i = TabularCPD('intel', 2, [[0.7], [0.3]])
cpd_g = TabularCPD('grade', 3, [[0.3, 0.05, 0.9, 0.5], 
                                [0.4, 0.25,0.08, 0.3], 
                                [0.3, 0.7, 0.02, 0.2]],
               ['intel', 'diff'], [2, 2])
student.add_cpds(cpd_d, cpd_i, cpd_g)

In [3]:

def get_value_from_cpd(cpd_values,random_value):
    cpd_value_line = []
    current = 0;
    for i,cpd_value in enumerate(cpd_values):
        current += cpd_value
        if random_value<= current:
            return i

In [4]:
def get_value_from_cpd_evidence(cpd_values,evidence,random_value):
    k_length = len(cpd_values)
    cpd_value_line = []
    current = 0;
    for k in range(k_length):
        v = cpd_values[k][evidence[0]][evidence[1]]
        current += v
        if random_value<= current:
            return k
        #print(v)

In [5]:
model = student
sorted_order = list(nx.topological_sort(student))
sorted_order

['intel', 'diff', 'grade']

In [6]:
sample_size = 10

In [20]:
variable_count = len(student.nodes)
size = sample_size * variable_count
print(size)
r = [random.random() for _ in range(size)]
r

30


[0.3923109564219488,
 0.07121676119484877,
 0.5905351813493315,
 0.5258161018307295,
 0.21363155021381364,
 0.5708016657792119,
 0.3192765141356242,
 0.6670552762044116,
 0.03254147395669382,
 0.5023009739013418,
 0.27724001675506393,
 0.7432221285498595,
 0.14267794596242733,
 0.19661142590490266,
 0.7208682838505683,
 0.73364567530697,
 0.02141542779147032,
 0.8977773906108545,
 0.33846426798311846,
 0.7396951291051057,
 0.9428493343833309,
 0.4195706653713486,
 0.45867087448363186,
 0.5102677992886416,
 0.8770653548333583,
 0.3450585079522328,
 0.927034708766155,
 0.1643619821323734,
 0.44779733621204376,
 0.32412581423142517]

In [8]:
k = 0
temp = dict()
for i in range(sample_size):
    for node in sorted_order:
            cpd = model.get_cpds(node)
            evidence = cpd.variables[:0:-1]
            #print(evidence)  
            v = None
            if evidence:
                #
                evidence_value = []
                for ev in evidence:
                    evidence_value.append(temp[ev])   
                v = get_value_from_cpd_evidence(cpd_g.values,evidence_value,r[k])
                
            else:
                #no evidence case
                v = get_value_from_cpd(cpd.values,r[k])
            
            temp[cpd.variables[0]] = v
            k+=1
    print(temp)

{'intel': 0, 'diff': 1, 'grade': 0}
{'intel': 0, 'diff': 1, 'grade': 1}
{'intel': 0, 'diff': 1, 'grade': 1}
{'intel': 0, 'diff': 1, 'grade': 0}
{'intel': 0, 'diff': 1, 'grade': 0}
{'intel': 0, 'diff': 0, 'grade': 2}
{'intel': 0, 'diff': 0, 'grade': 1}
{'intel': 1, 'diff': 0, 'grade': 2}
{'intel': 1, 'diff': 1, 'grade': 0}
{'intel': 1, 'diff': 1, 'grade': 2}


In [87]:
def prior_sampling(model,sample_size):
    
    sorted_order = list(nx.topological_sort(model))
    variable_count = len(model.nodes)
    size = sample_size * variable_count
    #print(size)
    r = [random.random() for _ in range(size)]
    k = 0
    temp = dict()
    sampled = []
    
    for i in range(sample_size):
        for node in sorted_order:
            cpd = model.get_cpds(node)
            evidence = cpd.variables[:0:-1]
            #print(evidence)  
            v = None
            if evidence:
                #
                evidence_value = []
                for ev in evidence:
                    evidence_value.append(temp[ev])   
                v = get_value_from_cpd_evidence(cpd_g.values,evidence_value,r[k])
                
            else:
                #no evidence case
                v = get_value_from_cpd(cpd.values,r[k])
            
            temp[cpd.variables[0]] = v
            k+=1
            
        sampled.append(temp.copy())
        print(temp)
    return sampled

In [90]:
sample = prior_sampling(student,10)

{'intel': 1, 'diff': 0, 'grade': 2}
{'intel': 0, 'diff': 0, 'grade': 2}
{'intel': 0, 'diff': 1, 'grade': 0}
{'intel': 1, 'diff': 0, 'grade': 1}
{'intel': 0, 'diff': 1, 'grade': 1}
{'intel': 1, 'diff': 1, 'grade': 0}
{'intel': 0, 'diff': 0, 'grade': 1}
{'intel': 1, 'diff': 0, 'grade': 1}
{'intel': 0, 'diff': 0, 'grade': 2}
{'intel': 0, 'diff': 0, 'grade': 1}


In [91]:
sample

[{'intel': 1, 'diff': 0, 'grade': 2},
 {'intel': 0, 'diff': 0, 'grade': 2},
 {'intel': 0, 'diff': 1, 'grade': 0},
 {'intel': 1, 'diff': 0, 'grade': 1},
 {'intel': 0, 'diff': 1, 'grade': 1},
 {'intel': 1, 'diff': 1, 'grade': 0},
 {'intel': 0, 'diff': 0, 'grade': 1},
 {'intel': 1, 'diff': 0, 'grade': 1},
 {'intel': 0, 'diff': 0, 'grade': 2},
 {'intel': 0, 'diff': 0, 'grade': 1}]